# Soustraction d'image moyenne sur une vidéo

## Etape 1 : Choix de la vidéo

In [3]:
import ipywidgets as widgets
import os


alldir=os.listdir(os.getcwd()+"/Data")
#listeVideo=[filename for filename in allfiles if  filename[-4:] in [".mp4",".avi",".mov",".3gp"]]
listeVideo=[filename for filename in alldir]

nomVideo = widgets.Dropdown(options=listeVideo,
    description='Choisissez une vidéo :',
    disabled=False,
)
display(nomVideo)

out = widgets.Output()


def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        out.clear_output()
        with out :
            print("Vous avez choisi %s" % change['new'])
nomVideo.observe(on_change)
display(out)

Dropdown(description='Choisissez une vidéo :', options=('Arc_Compil_Tir', 'Billard_Coup_1', 'Billard_Coup_2', …

Output()

## Etape 2 : Soustraction de fond

In [8]:
allfiles=os.listdir(os.getcwd()+"/Data/"+nomVideo.value)
file=[name for name in allfiles if (name.startswith(nomVideo.value) and name[-4:] in [".mp4",".avi",".mov",".3gp"])][0]
#file=nomVideo.value
print(file)

# lecture de la vidéo

from IPython.display import HTML

HTML("""
<video width="320" height="240" controls>
  <source src="Vidéos/file" type="video/mp4">
</video>
""")


##Récupération des paramètres dans un csv

startSecond=0
stopSecond=-1
tailleIntervalle=5
posIm=3

import cv2,numpy


def video2listImage(adresseVideo,staSec,stoSec): # Renvoie une liste d'image sous forme d'array
    video = cv2.VideoCapture(adresseVideo)
    videoFPS = video.get(cv2.CAP_PROP_FPS)
    print(videoFPS)
    
    # Première étape, aller au time code de départ
    continuer=True
    compteur=0
    while continuer:
        isRead = video.read()
        if not isRead:
            continuer = False
            print("Time code de départ trop grand")
            return None
        else:
            compteur+=1
            if (compteur/videoFPS>staSec):
                continuer = False
    
    compteur-=1
    if staSec!=0:
        print("Time code de début atteint",compteur/videoFPS)
    
    imlist=[]
    continuer = True
    
    while continuer:
        isRead , frame = video.read()  # A-t-on lu une image et si oui cette image
        if not isRead:
            continuer = False
        else:
            #frameTimeCode = compteur/videoFPS
            imlist.append(frame)
            compteur+=1
            
            if (stoSec!=-1): #Si on ne va pas jusqu'à la fin de la vidéo
                if(compteur/videoFPS>stoSec):
                    print("Time code de fin atteint",compteur/videoFPS)
                    continuer=False
    
    if (compteur/videoFPS<stoSec):
        print("Time code de fin trop grand")
        return None
    
    return imlist,videoFPS

def subImageMoyArr(imListArr,PosIm): # Renvoie un tableau OpenCV
    n=len(imListArr)
    imCentre=imListArr[PosIm]
    imMoy=calcImageMoyenneArr(imListArr)
    imSub = cv2.subtract(imCentre,imMoy)
    return imSub

def calcImageMoyenneArr(imListArr): # Renvoie array
    w,h=len(imListArr[0]), len(imListArr[0][0])
    arr=numpy.zeros((h,w,4),numpy.float)
    images = numpy.array([image for image in imListArr])
    # Round values in array and cast as 8-bit integer
    arr=numpy.array(numpy.mean(images,axis=(0)),dtype=numpy.uint8)
    return arr

def SubImageMoySurListe(imList,tailleMoy,PosIm):
    imlistMoy = [0]*tailleMoy
    imlistSubMoy = [0]*(len(imList)-tailleMoy+1)
    for i in range(PosIm,len(imList)-(tailleMoy-PosIm)):
        # On selectionne les images autour de celle traitée
        for j in range(tailleMoy):
            imlistMoy[j]=imList[j+i-PosIm]
        imlistSubMoy[i-PosIm]=subImageMoyArr(imlistMoy,PosIm)
    return imlistSubMoy



imList, FPS = video2listImage("Vidéos/"+file,startSecond,stopSecond) # Tableau d'array
if (imList!=None):
    imListSubMoy=SubImageMoySurListe(imList, tailleIntervalle,posIm) # Les mêmes images, auxquelles on a enlevé les images moyennes
    print('OK')



#Enregistrement de la vidéo
def listImage2Video(imList,writeFPS,nomFichier):
    height,width,layers=imList[0].shape
    # choose codec according to format needed
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video=cv2.VideoWriter(nomFichier, fourcc, writeFPS,(width,height))
    for img in imList:
        video.write(img)
    return video
   
videoWriteFile="VidéosSUB/SUB "+file
videoSub = listImage2Video(imListSubMoy,FPS, videoWriteFile)
videoSub.release()
print("Done")

from IPython.display import HTML

HTML("""
<video width="320" height="240" controls>
  <source src="videoWriteFile" type="video/mp4">
</video>
""")

TT_echanges_1.mov
